In [2]:
import os
from glob import glob
os.listdir("../data/facades_data")

['facades_data']

In [3]:
img_files = glob("../data/facades_data/facades_data/*.jpg")
map_files = glob("../data/facades_data/facades_data/*.bmp")
img_files[:2],map_files[:2],os.path.basename(img_files[0])

(['../data/facades_data/facades_data\\cmp_b0001.jpg',
  '../data/facades_data/facades_data\\cmp_b0002.jpg'],
 ['../data/facades_data/facades_data\\cmp_b0001.bmp',
  '../data/facades_data/facades_data\\cmp_b0002.bmp'],
 'cmp_b0001.jpg')

In [7]:
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
label = Image.open(map_files[0]) 
label = np.array(label)
label.shape

(565, 300)

In [8]:
np.unique(label.flatten())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11], dtype=uint8)

In [9]:
import torch.nn.functional as F
import torch
label_ohe = torch.tensor(label).long() 
label_ohe = F.one_hot(label_ohe,num_classes=12)

label_ohe.shape

torch.Size([565, 300, 12])

In [26]:
map_transforms = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
        ])
label = Image.open(map_files[0]) 
print(label.size)        
label_tr = map_transforms(label).squeeze(0)     
label_tr.shape

(300, 565)


torch.Size([256, 256])

In [27]:
  
label_tr = (label_tr*255).long()

In [32]:
label_ohe = F.one_hot(label_tr,num_classes=12)
label_ohe.shape

torch.Size([256, 256, 12])

In [33]:
torch.max(label_ohe),torch.min(label_ohe)

(tensor(1), tensor(0))

In [18]:
label_ohe.permute(2,0,1).shape

torch.Size([12, 565, 300])

In [30]:
img_transforms = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
img = Image.open(img_files[0]).convert("RGB")
img = img_transforms(img)
img.shape

torch.Size([3, 256, 256])

In [35]:
map_transforms = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
        ])
label = Image.open(img_files[0].replace("jpg","bmp"))
        
label = map_transforms(label).long().squeeze(0)
print(label.shape)
label_ohe = F.one_hot(label,num_classes=12).permute(2,0,1)
label_ohe.shape

torch.Size([256, 256])


torch.Size([12, 256, 256])

In [62]:
class Config():
    IMAGE_HEIGHT = 256
    IMAGE_WIDTH = 256
    BATCH_SIZE = 4
    LATENT_DIM = 256
    DEVICE = "cpu"
    NUM_CLASSES = 12
cfg = Config()
class GauGanDataset:
    def __init__(self,img_paths):
        self.img_paths = img_paths
        self.img_transforms = transforms.Compose([
            transforms.Resize((cfg.IMAGE_HEIGHT,cfg.IMAGE_WIDTH)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.map_transforms = transforms.Compose([
            transforms.Resize((cfg.IMAGE_HEIGHT,cfg.IMAGE_WIDTH)),
            transforms.ToTensor(),
        ])
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self,index):
        image = Image.open(self.img_paths[index]).convert("RGB")
        label = Image.open(self.img_paths[index].replace("jpg","bmp"))
        
        image = self.img_transforms(image)
        label = self.map_transforms(label).long().squeeze(0)
        label_ohe = F.one_hot(label,num_classes=cfg.NUM_CLASSES).permute(2,0,1)
        # print("here",index,image.shape,label_ohe.shape)
        return {
            "image":image,
            "segmentation_map":label_ohe
        }
        

In [65]:
train_dataset=GauGanDataset(img_files)
train_loader=torch.utils.data.DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        num_workers=0
    )
for batch in train_loader:
    print(batch["image"].shape)
    break


torch.Size([4, 3, 256, 256])


str

TypeError: string indices must be integers